In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn import metrics

In [3]:
import torch
from torchvision import transforms, datasets

data_transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        # transforms.RandomSizedCrop(224),
#         transforms.RandomHorizontalFlip(),
#         transforms.RandomVerticalFlip(),
#         transforms.RandomRotation(degrees=180),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

train_fer = datasets.ImageFolder(root='/tf/data/Quan/fer2013/data/train', transform=data_transform)
val_fer = datasets.ImageFolder(root='/tf/data/Quan/fer2013/data/val', transform=data_transform)
test_fer = datasets.ImageFolder(root='/tf/data/Quan/fer2013/data/test', transform=data_transform)

batch_size = 64
train_loader = torch.utils.data.DataLoader(train_fer,
                                             batch_size=batch_size, shuffle=True,
                                             num_workers=8)
val_loader = torch.utils.data.DataLoader(val_fer,
                                             batch_size=batch_size, shuffle=True,
                                             num_workers=8)
test_loader = torch.utils.data.DataLoader(test_fer,
                                             batch_size=batch_size, shuffle=True,
                                             num_workers=8)

In [4]:
class WeightFeature(nn.Module):

    def __init__(self, feature_depth, target_depth):
        super(WeightFeature, self).__init__()

        self.target_depth = target_depth
#         self.conv1 = nn.Conv2d(feature_depth, feature_depth, kernel_size=feature_size, padding=0)
#         self.bn = nn.BatchNorm2d(feature_depth)
        self.fc1 = nn.Linear(feature_depth, target_depth)
        self.bn1 = nn.BatchNorm1d(target_depth)
        self.bn2 = nn.BatchNorm2d(target_depth)
        
    def forward(self, x1, x2):
        '''
        x1 -> GAP -> FC -> * with x2-> x2 --> out
        x2 ----------------------------|
        '''
        x1 = nn.AvgPool2d(kernel_size=(x1.size(-2), x1.size(-1)))(x1)
#         x1 = self.conv1(x1)
#         x1 = self.bn(x1)
#         x1 = nn.ReLU()(x1)
        x1 = nn.Flatten()(x1)

        x1 = self.fc1(x1)
        x1 = nn.Dropout(0.6)(x1)
        x1 = self.bn1(x1)
        x1 = nn.Sigmoid()(x1)
#         x1 = nn.ReLU()(x1)
        x_out = x1.unsqueeze(-1).unsqueeze(-1)
        # print(x1.size())
        # x2 = self.conv1(x2)

        x_out = torch.mul(x2, x_out)
        x_out = x_out + x2 
        x_out = self.bn2(x_out)

        return x_out

In [5]:
class SVblock(nn.Module):
    def __init__(self, in_neurons, nb_neurons, cardinality=4, kernel_size=(3,3), batch_norm=True, activation='relu', **kwargs):
        '''
    in_neurons: C_in
    nb_neurons: n_filters of conv2d
    '''
        super(SVblock, self).__init__()
        
        self.device = torch.device('cpu') if not 'device' in kwargs else kwargs['device']
        self.index = kwargs['index'] if 'index' in kwargs else None
        self.card = cardinality
        self.n_set = nb_neurons // self.card
        
        self.wf = WeightFeature(nb_neurons, nb_neurons)
        
        self.pre_convs = nn.Sequential(nn.Conv2d(in_neurons, nb_neurons, kernel_size=kernel_size, padding=1, stride=1),
                                       nn.BatchNorm2d(nb_neurons),
                                       nn.ReLU(inplace=True))
        
        self.convs = []
        for i in range(self.n_set):
            self.convs.append(nn.Sequential(nn.Conv2d(nb_neurons, self.card, kernel_size=(1,1), padding=0, stride=1),
                                            nn.BatchNorm2d(self.card), nn.ReLU(inplace=True),
                                            nn.Conv2d(self.card, self.card, kernel_size=kernel_size, padding=1, stride=1),
                                            nn.BatchNorm2d(self.card), nn.ReLU(inplace=True),
                                            nn.Conv2d(self.card, nb_neurons, kernel_size=(1,1), padding=0, stride=1),
                                            nn.BatchNorm2d(nb_neurons), nn.ReLU(inplace=True)).to(self.device))      
        
        
    def forward(self, x):
        '''
        x: single output layer of list 
        '''

        x = self.pre_convs(x)
        
        out = self.convs[0](x)
        for i in range(1, len(self.convs)):
            out = out + self.convs[i](x)
            
        x_out = self.wf(x, out)

        return x_out

In [6]:
class SVGG(nn.Module):

    def __init__(self, **kwargs):
        super(SVGG, self).__init__()

        self.device = torch.device('cpu') if not 'device' in kwargs else kwargs['device']

        self.sv0 = SVblock(1,64).to(self.device)
#         self.sv00 = SVblock(64,64).to(self.device)
        self.sv1 = SVblock(64, 128).to(self.device)
#         self.sv11 = SVblock(128, 128).to(self.device)
        self.sv2 = SVblock(128, 256).to(self.device)
#         self.sv22 = SVblock(256, 256).to(self.device)
        self.sv3 = SVblock(256, 512).to(self.device)
#         self.sv33 = SVblock(512, 512).to(self.device)
        self.sv4 = SVblock(512, 1024, feature_size=3).to(self.device)
#         self.sv44 = SVblock(1024, 1024).to(self.device)

        self.bn_preout = nn.BatchNorm1d(3008)
        self.fc1 = nn.Linear(3008, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 7)
        self.bn2 = nn.BatchNorm1d(7)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x0 = self.sv0(x)
#         x0 = self.sv00(x0)
        x0 = nn.MaxPool2d(kernel_size=(2,2), stride=2)(x0)
        
        x1 = self.sv1(x0)
#         x1 = self.sv11(x1)
        x1 = nn.MaxPool2d(kernel_size=(2,2), stride=2)(x1)
        
        x2 = self.sv2(x1)
#         x2 = self.sv22(x2)
        x2 = nn.MaxPool2d(kernel_size=(2,2), stride=2)(x2)
        
        x3 = self.sv3(x2)
#         x3 = self.sv33(x3)
        x3 = nn.MaxPool2d(kernel_size=(2,2), stride=2)(x3)
        
        x4 = self.sv4(x3)
#         x = self.sv44(x)
        x4 = nn.MaxPool2d(kernel_size=(2,2), stride=2)(x4)
        
        x4 = nn.AvgPool2d(kernel_size=(x4.size(-2), x4.size(-1)))(x4)
        x4 = nn.Flatten()(x4)
        
#         x3 = nn.AvgPool2d(kernel_size=(x3.size(-2), x3.size(-1)))(x3)
#         x3 = nn.Flatten()(x3)
        
#         x2 = nn.AvgPool2d(kernel_size=(x2.size(-2), x2.size(-1)))(x2)
#         x2 = nn.Flatten()(x2)
        
#         x1 = nn.AvgPool2d(kernel_size=(x1.size(-2), x1.size(-1)))(x1)
#         x1 = nn.Flatten()(x1)
        
#         x0 = nn.AvgPool2d(kernel_size=(x0.size(-2), x0.size(-1)))(x0)
#         x0 = nn.Flatten()(x0)
        
#         x_out = torch.cat([x4, x3, x2, x1, x0], dim=1)

        x_out = torch.cat([gap0, gap1, gap2, gap3, gap4, x4], dim=1)
        

        x_out = self.fc1(x_out)
        x_out = self.bn1(x_out)
        x_out = nn.Dropout(0.6)(x_out)
        x_out = self.fc2(x_out)
        x_out = self.bn2(x_out)

        return x_out


In [ ]:
device = torch.device('cuda')
print(device)

model = SVGG(device=device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# model.train()

# model = model.to(device)

In [ ]:
model = SVGG()
model(torch.rand(128, 1, 48, 48))